<a href="https://colab.research.google.com/github/jpraveenkanna/Style_generator-Video_and_Image/blob/master/Style_Transfer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Dependency
!pip install flask-ngrok

In [0]:
#Libraries

#colab specific
%tensorflow_version 2.x
from google.colab.patches import cv2_imshow

import cv2
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import json
from flask import Flask, render_template,request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)



#global
#style_model = "/tmp/tfhub_modules/f843094219bf78a99e8ea6c8d71f1bc74f07101a"
style_model = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(style_model)

def run_inference(source_image,source_style):
  style_input = source_style
  input_image = source_image


  style_height = style_width = 256
  style_img = style_input.astype(np.float32)[np.newaxis, ...]
  if style_img.max() > 1.0:
      style_img = style_img / 255.
  if len(style_img.shape) == 3:
    style_img = tf.stack([style_img, style_img, style_img], axis=-1)
  style = tf.image.resize(style_img, (style_height,style_width), preserve_aspect_ratio=True)

  #maintaining original size of input image
  image_size=(input_image.shape[1],input_image.shape[0])
  img = input_image.astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
      img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  outputs = hub_module(tf.constant(img), tf.constant(style))
  stylized_image = outputs[0]
  img_background = cv2.normalize(src=stylized_image[0].numpy(), dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
  final_style = cv2.resize(img_background,(input_image.shape[1],input_image.shape[0]),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)


  #return stylized image
  data = json.dumps({"style_image": final_style.tolist()})
  return data


@app.route('/')
def index():
  return "OK"


@app.route("/style", methods=["POST"])
def style():
  if request.method == 'POST':
    data = request.get_json()
    images = json.loads(data)
    style_img = np.asarray(images["style_image"])
    source_img = np.asarray(images["source_image"])
    json_response = run_inference(source_img,style_img)

    return json_response

if __name__ == "__main__":
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://53721dcc.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
